In [2]:
import google.generativeai as genai
from settings import get_settings
from connectors.google import GeminiLLM

/Users/lioche/Documents/@TROOPLE/API_DOCUMENT_INTELLIGENCE/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
llm = GeminiLLM()

In [4]:
with open("../311-2023-016324.pdf", "rb") as f:
    file = f.read()

In [5]:
from io import BytesIO
by = BytesIO(file)

In [30]:
ufile = llm.upload_file(by, "application/pdf", True)

{'name': 'files/ab6lh9gjsnax', 'mime_type': 'application/pdf', 'size_bytes': '267999', 'create_time': '2024-10-15T10:43:00.311519Z', 'update_time': '2024-10-15T10:43:00.311519Z', 'expiration_time': '2024-10-17T10:43:00.292226005Z', 'sha256_hash': 'M2Y0OGEzMjk4OTMwOGE0ZDNjYzI1Y2FjNGJkNDNjMWEwZDFjM2RiMTMxZTk0OGU1MGRlNzFmNTA1N2Q5MGFjNg==', 'uri': 'https://generativelanguage.googleapis.com/v1beta/files/ab6lh9gjsnax', 'state': 'ACTIVE', 'display_name': ''}


In [31]:
ufile

genai.File({
    'name': 'files/ab6lh9gjsnax',
    'display_name': '',
    'mime_type': 'application/pdf',
    'sha256_hash': 'M2Y0OGEzMjk4OTMwOGE0ZDNjYzI1Y2FjNGJkNDNjMWEwZDFjM2RiMTMxZTk0OGU1MGRlNzFmNTA1N2Q5MGFjNg==',
    'size_bytes': '267999',
    'state': 'ACTIVE',
    'uri': 'https://generativelanguage.googleapis.com/v1beta/files/ab6lh9gjsnax',
    'create_time': '2024-10-15T10:43:00.311519Z',
    'expiration_time': '2024-10-17T10:43:00.292226005Z',
    'update_time': '2024-10-15T10:43:00.311519Z'})

In [23]:
genai.get_file("files/obs7lne9hxx")

PermissionDenied: 403 You do not have permission to access the File obs7lne9hxx or it may not exist.

In [3]:
genai.configure(api_key=get_settings().GOOGLE_API_KEY.get_secret_value())

In [4]:
model = genai.GenerativeModel('models/gemini-1.5-flash')
model.count_tokens(get_settings().SYSTEM_PROMPT)

total_tokens: 230

In [19]:
def get_prices(input: int, output: int):
    prices = {
        "gemini-1.5-flash": (0.075, 0.3),
        "gemini-1.5-pro": (1.25, 5.0),
    }
    outputs = {}
    for model, (min_price, max_price) in prices.items():
        min_price = min_price / 1_000_000
        max_price = max_price / 1_000_000
        outputs[model] = round(min_price * input + max_price * output, 6)
    return outputs


In [22]:
get_prices(4716, 3341)

{'gemini-1.5-flash': 0.001356, 'gemini-1.5-pro': 0.0226}

In [23]:
from PyPDF2 import PdfReader, PdfWriter
pdf = PdfReader(by)

In [28]:
def split_pdf_group_of_pages(pdf, pages_per_group: int = 2) -> list[BytesIO]:
    splits = []
    for i in range(0, len(pdf.pages), pages_per_group):
        writer = PdfWriter()
        bio = BytesIO()
        idx_start = i
        idx_end = min(i + pages_per_group, len(pdf.pages))
        for page in pdf.pages[idx_start:idx_end]:
            writer.add_page(page)
        writer.write(bio)
        with open(f"split_{idx_start}_{idx_end}.pdf", "wb") as f:
            f.write(bio.getvalue())
        splits.append(bio)
    return splits

In [29]:
split_pdf_group_of_pages(pdf, 2)

[<_io.BytesIO at 0x12086d5d0>, <_io.BytesIO at 0x1208ae6b0>]

In [30]:
print("""  "Neerlegging-Dépôt: 29/04/2024\nRegist.-Enregistr.: 17/05/2024\nN°: 187727/CO/311\nPARITAIR COMITE VOOR DE GROTE\nKLEINHANDELSZAKEN\nPC 311\nCOLLECTIEVE ARBEIDSOVEREENKOMST VAN 18\nAPRIL 2024BETREFFENDE HET STELSEL VAN\nWERKLOOSHEID MET BEDRIJFSTOESLAG\n(Juli-oktober 2025)\nHOOFDSTUK 1 - TOEPASSINGSGEBIED\nArtikel 1-§ 1. Deze collectieve\narbeidsovereenkomst is van toepassing op de\nwerkgevers en op de werknemers van de\nondernemingen welke ressorteren onder het\nParitair comité voor de grote kleinhandelszaken\n(PC 311).\n§ 2. Met \"werknemers\" worden de mannelijke\nen vrouwelijke werknemers bedoeld.\nHOOFDSTUK 2 - WERKLOOSHEID MET\nBEDRIJFSTOESLAG VOOR WERKNEMERS MET\nEEN BEROEPSLOOPBAAN VAN 33 JAAR IN EEN\nZWAAR BEROEP\nArtikel 2 - § 1. Onverminderd de toepassing van\nde collectieve arbeidsovereenkomst nr. 17 van\n19 december 1974 \"tot invoering van een\nregeling van aanvullende vergoeding ten gunste\nvan sommige bejaarde werknemers indien zij\nworden ontslagen\" en in uitvoering van de\ntoekomstige collectieve arbeidsovereenkomst\nvan de NAR, wordt aan de ontslagen\nwerknemers, die voldoen aan de voorwaarden\nvastgesteld in de volgende paragrafen, het\nvoordeel van het stelsel van werkloosheid met\nbedrijfstoeslag toegekend.\n§ 2. Werknemers die ontslagen worden moeten\n60 jaar of ouder zijn:\nOp het ogenblik van het einde van de\narbeidsovereenkomst;\nEn ten laatste op 30 juni 2025.\n§ 3. De loopbaanvoorwaarde van 33 jaar moet\nvervuld zijn op het ogenblik van de het einde\nvan de arbeidsovereenkomst.\nHOOFDSTUK 3 - WERKLOOSHEID MET\nBEDRIJFSTOESLAG VOOR WERKNEMERS MET\nEEN BEROEPSLOOPBAAN VAN 35 JAAR IN EEN\nZWAAR BEROEP\nArtikel 3-§ 1. Onverminderd de toepassing van\nde collectieve arbeidsovereenkomst nr. 17 van\n19 december 1974 \"tot invoering van een\nregeling van aanvullende vergoeding ten gunste\nvan sommige bejaarde werknemers indien zij\nworden ontslagen\" en in uitvoering van artikel\n3, § 3 van het koninklijk besluit van 3 mei 2007\ntot regeling van het stelsel van werkloosheid\nmet bedrijfstoeslag en de toekomstige\ncollectieve arbeidsovereenkomst van de NAR,\nwordt aan de ontslagen werknemers, die\nvoldoen aan de voorwaarden vastgesteld in de\nvolgende paragrafen, het voordeel van het\nstelsel van werkloosheid met bedrijfstoeslag\ntoegekend.\n§ 2. Werknemers die ontslagen worden moeten\n60 jaar of ouder zijn:\nOp het ogenblik van het einde van de\narbeidsovereenkomst;\nEn ten laatste op 30 juni 2025.\n§ 3. De loopbaanvoorwaarde van 35 jaar moet\nvervuld zijn op het ogenblik van het einde van\nde arbeidsovereenkomst.\nHOOFDSTUK 4 - WERKLOOSHEID MET\nBEDRIJFSTOESLAG VOOR WERKNEMERS MET\nEEN BEROEPSLOOPBAAN VAN 40 JAAR\nArtikel 4 - § 1. Onverminderd de toepassing van\nde collectieve arbeidsovereenkomst nr. 17 van\n19 december 1974 \"tot invoering van een\nregeling van aanvullende vergoeding ten gunste\nvan sommige bejaarde werknemers indien zij\nworden ontslagen\" en in uitvoering van de\ntoekomstige collectieve arbeidsovereenkomst\nvan de NAR, wordt aan de ontslagen\nCOMMISSION PARITAIRE POUR DES GRANDES\nENTREPRISES DE VENTE AU DETAIL\nCP 311\nCONVENTION COLLECTIVE DE TRAVAIL DU 18\nAVRIL 2024 RELATIVE AU RÉGIME DE\nCHÔMAGE AVEC COMPLÉMENT D'ENTREPRISE\n(Juillet-octobre 2025)\nCHAPITRE 1 - CHAMP D'APPLICATION\nArticle 1 - § 1. La présente convention collective\nde travail s'applique aux employeurs et aux\ntravailleurs des entreprises relevant de la\nCommission paritaire des grandes entreprises\nde vente au détail (CP 311).\n§ 2. Par \"travailleurs\" sont visés les travailleurs\nmasculins et féminins.\nCHAPITRE 2 - CHÔMAGE AVEC COMPLÉMENT\nD'ENTREPRISE POUR LES TRAVAILLEURS AVEC\n33 ANS DE CARRIÈRE DANS UN MÉTIER LOURD\nArticle 2 - § 1. Sans préjudice de l'application de\nla convention collective de travail n°17 du 19\ndécembre 1974 << instituant un régime\nd'indemnité complémentaire pour certains\ntravailleurs âgés en cas de licenciement >> et en\nexécution de la future convention collective de\ntravail du CNT, il est octroyé aux travailleurs\nlicenciés qui satisfont aux conditions énoncées\ndans les paragraphes suivants, l'avantage du\nrégime de chômage avec complément\nd'entreprise.\n§ 2. Les travailleurs licenciés doivent avoir\natteint l'âge de 60 ans au moins :\nÀ la fin de leur contrat de travail;\nEt au plus tard le 30 juin 2025.\n§ 3. La condition de carrière de 33 ans doit être\nremplie au moment de la fin du contrat de\ntravail.\nCHAPITRE 3 - CHÔMAGE AVEC COMPLÉMENT\nD'ENTREPRISE POUR LES TRAVAILLEURS AVEC\n35 ANS DE CARRIÈRE DANS UN MÉTIER LOURD\nArticle 3 - § 1. Sans préjudice de l'application de\nla convention collective de travail n°17 du 19\ndécembre 1974 << instituant un régime\nd'indemnité complémentaire pour certains\ntravailleurs âgés en cas de licenciement >> et en\nexécution de l'article 3, § 3 de l'arrêté royal du\n3 mai 2007 fixant le régime de chômage avec\ncomplément d'entreprise et de la future\nconvention collective de travail du, il est\noctroyé aux travailleurs licenciés qui satisfont\naux conditions énoncées dans les paragraphes\nsuivants, l'avantage du régime de chômage avec\ncomplément d'entreprise.\n§ 2. Les travailleurs licenciés doivent avoir\natteint l'âge de 60 ans au moins :\nÀ la fin de leur contrat de travail;\nEt au plus tard le 30 juin 2025.\n§ 3. La condition de carrière de 35 ans doit être\nremplie au moment de la fin du contrat de\ntravail.\nCHAPITRE 4 - CHÔMAGE AVEC COMPLÉMENT\nD'ENTREPRISE POUR LES TRAVAILLEURS AVEC\n40 ANS DE CARRIÈRE\nArticle 4 - § 1. Sans préjudice de l'application de\nla convention collective de travail n°17 du 19\ndécembre 1974 << instituant un régime\nd'indemnité complémentaire pour certains\ntravailleurs âgés en cas de licenciement >> et en\nexécution de la future convention collective de\ntravail du CNT, il est octroyé aux travailleurs\nlicenciés qui satisfont aux conditions énoncées\nPC 311 WERKLOOSHEID MET BEDRIJFSTOESLAG\n1/4\nCP 311 CHÔMAGE AVEC COMPLÉMENT D'ENTREPRISE\nPC 311 WERKLOOSHEID MET BEDRIJFSTOESLAG\n2/4\nCP 311 CHÔMAGE AVEC COMPLÉMENT D'ENTREPRISE\n==End of OCR for page 2== ==Start of OCR for page 1==\nwerknemers, die voldoen aan de voorwaarden\nvastgesteld in de volgende paragrafen, het\nvoordeel van het stelsel van werkloosheid met\nbedrijfstoeslag toegekend.\n§ 2. Werknemers die ontslagen worden moeten\n60 jaar of ouder zijn:\nOp het ogenblik van het einde van de\narbeidsovereenkomst;\nEn ten laatste op 30 juni 2025.\n§ 3. De loopbaanvoorwaarde van 40 jaar moet\nvervuld zijn op het ogenblik van het einde van\nde arbeidsovereenkomst.\nHOOFDSTUK 5 - OVERGANG TIJDSKREDIET -\nWERKLOOSHEID MET BEDRIJFSTOESLAG\nArtikel 5 - Voor de werknemers die genieten van\neen vermindering van de arbeidsprestaties tot\neen 4/5de of tot een halftijdse betrekking en die\noverstappen in het stelsel van werkloosheid\nmet bedrijfstoeslag, wordt de aanvullende\nvergoeding berekend op basis van het bruto\nmaandloon dat de werknemer zou verdienen\nindien hij zijn arbeidsprestaties niet zou\nverminderd hebben en de\nwerkloosheidsuitkeringen overeenstemmend\nmet het arbeidsregime in voege voor de\naanvang van het tijdskrediet.\nHOOFDSTUK 6 - WERKHERVATTING\nArtikel 6 - Bij werkhervatting van de werkloze\nmet bedrijfstoeslag binnen de voorwaarden van\nartikelen 114 en 115 van de Wet van 27\ndecember 2006 houdende diverse bepalingen,\nnamelijk als werknemer in loondienst bij een\nandere werkgever dan de schuldenaar van de\nwerkloosheid met bedrijfstoeslag of voor\nrekening van een werkgever die niet behoort\ntot dezelfde technische bedrijfseenheid als de\nwerkgever die hem heeft ontslagen, of als\nzelfstandige in hoofdberoep, voor zover deze\nactiviteit niet wordt uitgeoefend bij de\ndans les paragraphes suivants, l'avantage du\nrégime de chômage avec complément\nd'entreprise.\n§ 2. Les travailleurs licenciés doivent avoir\natteint l'âge de 60 ans au moins :\nÀ la fin de leur contrat de travail;\nEt au plus tard le 30 juin 2025.\n§ 3. La condition de carrière de 40 ans doit être\nremplie au moment de la fin du contrat de\ntravail.\nCHAPITRE 5 - PASSAGE CREDIT-TEMPS -\nCHÔMAGE AVEC COMPLÉMENT D'ENTREPRISE\nArticle 5 - Pour les travailleurs bénéficiant d'une\nréduction des prestations de travail à 4/5ième\ntemps ou à mi-temps et qui entrent dans le\nrégime de chômage avec complément\nd'entreprise, l'indemnité complémentaire est\ncalculée sur base du salaire mensuel brut que le\ntravailleur aurait gagné s'il n'avait pas réduit ses\nprestations de travail et des allocations de\nchômage correspondant au régime de travail\navant la prise du crédit-temps.\nCHAPITRE 6 - REPRISE DE TRAVAIL\nArticle 6 - En cas de reprise de travail par le\nchômeur avec complément d'entreprise dans\nles conditions des articles 114 et 115 de la Loi\ndu 27 décembre 2006 portant dispositions\ndiverses, notamment comme salarié auprès\nd'un autre employeur que le débiteur du\nchômage avec complément d'entreprise ou\nauprès d'un employeur n'appartenant pas à la\nmême unité technique d'exploitation que\nl'employeur qui l'a licencié ou en qualité de\ntravailleur indépendant à titre principal à\ncondition qu'il n'exerce pas son activité pour le\nwerkgever die de werknemer heeft ontslagen of compte de l'employeur qui l'a licencié ou pour\nvoor rekening van een werkgever die behoort\ntot dezelfde technische bedrijfseenheid als de\nle compte d'un employeur appartenant à la\nmême unité technique d'exploitation que\nPC 311 WERKLOOSHEID MET BEDRIJFSTOESLAG\n3/4\nCP 311 CHÔMAGE AVEC COMPLÉMENT D'ENTREPRISE\n==End of OCR for page 1==\n\n==Start of OCR for page 2==\nwerkgever die hem heeft ontslagen, zal de\naanvullende vergoeding worden verder betaald.\nHOOFDSTUK 7 - BESCHIKBAARHEID\nl'employeur qui l'a licencié, l'indemnité\ncomplémentaire continuera à être payée.\nCHAPITRE 7 - DISPONIBILITE\nArticle 7 - En exécution de l'article 22, § 3,\nalinéa 5 de l'arrêté royal du 3 mai 2007 fixant le\nrégime de chômage avec complément\nd'entreprise et de CCT n°168 et 169 du CNT,\nArtikel 7 - In uitvoering van artikel 22, § 3, lid 5\nvan het koninklijk besluit van 3 mei 2007 tot\nregeling van het stelsel van werkloosheid met\nbedrijfstoeslag en de CAO's nr. 168 en 169 van\nde NAR wordt de leeftijd vermeld in artikel 22, § l'âge mentionné à l'article 22, § 3, alinéa 4, 1°\n3, lid 4, 1° op 62 jaar gebracht vanaf 1 juli 2023\ntot 31 december 2026.\nHOOFDSTUK 8 - SLOTBEPALINGEN\nest porté à 62 ans à partir du 1er juillet 2023\njusqu'au 31 décembre 2026.\nCHAPITRE 8 - DISPOSITIONS FINALES\nArtikel 8 - Deze collectieve\nArticle 8 - La présente convention collective de\narbeidsovereenkomst treedt in werking op 1 juli travail entre en vigueur le 1er juillet 2025 et\n2025 en houdt op van kracht te zijn op 31\noktober 2025.\ncesse d'être d'application le 31 octobre 2025.\nPC 311 WERKLOOSHEID MET BEDRIJFSTOESLAG\n4/4\nCP 311 CHÔMAGE AVEC COMPLÉMENT D'ENTREPRISE\n==End of OCR for page 2==\n\nThe document discusses the regulations for unemployment benefits with company supplement. \n\n**Key Elements:**\n\n* **Names:** No names are mentioned in the document.\n* **Numbers:**\n    * 4/5de\n    * 60 jaar\n    * 30 juni 2025\n    * 40 jaar\n    * 114 and 115\n    * 27 december 2006\n    * 168 and 169\n    * 1 juli 2023\n    * 31 december 2026\n    * 1 juli 2025\n    * 31 oktober 2025\n* **Dates:**\n    * 30 juni 2025\n    * 27 december 2006\n    * 3 mei 2007\n    * 1 juli 2023\n    * 31 december 2026\n    * 1 juli 2025\n    * 31 oktober 2025\n* **People Mentioned:** No people are mentioned in the document.\n* **Checkboxes:** No checkboxes are present.\n* **Phone Numbers:** No phone numbers are present. \n\n**Logical Consistency:**\n* The document is written in Dutch and French and is consistent throughout.\n*  There are no inconsistencies found. \n\n**Transcription:**\nThe provided document details regulations regarding unemployment benefits with company supplement. It outlines the eligibility criteria for workers who are laid off and meet certain age and work experience requirements. \n\nThe document also discusses the benefits of unemployment benefits for workers who have reduced their work hours and transition to unemployment benefits. \n\nIt includes information about the conditions for resuming work while receiving unemployment benefits and the requirements for employers and employees in such situations. \n\nFinally, the document clarifies the availability of benefits and the specific age requirements for beneficiaries. It also details the date of effect and termination of the collective labor agreement. \n""")

  "Neerlegging-Dépôt: 29/04/2024
Regist.-Enregistr.: 17/05/2024
N°: 187727/CO/311
PARITAIR COMITE VOOR DE GROTE
KLEINHANDELSZAKEN
PC 311
COLLECTIEVE ARBEIDSOVEREENKOMST VAN 18
APRIL 2024BETREFFENDE HET STELSEL VAN
WERKLOOSHEID MET BEDRIJFSTOESLAG
(Juli-oktober 2025)
HOOFDSTUK 1 - TOEPASSINGSGEBIED
Artikel 1-§ 1. Deze collectieve
arbeidsovereenkomst is van toepassing op de
werkgevers en op de werknemers van de
ondernemingen welke ressorteren onder het
Paritair comité voor de grote kleinhandelszaken
(PC 311).
§ 2. Met "werknemers" worden de mannelijke
en vrouwelijke werknemers bedoeld.
HOOFDSTUK 2 - WERKLOOSHEID MET
BEDRIJFSTOESLAG VOOR WERKNEMERS MET
EEN BEROEPSLOOPBAAN VAN 33 JAAR IN EEN
ZWAAR BEROEP
Artikel 2 - § 1. Onverminderd de toepassing van
de collectieve arbeidsovereenkomst nr. 17 van
19 december 1974 "tot invoering van een
regeling van aanvullende vergoeding ten gunste
van sommige bejaarde werknemers indien zij
worden ontslagen" en in uitvoering van de
toekomstige collectieve 